In [ ]:
import cupy as cp
import numpy as np
import cupyx.scipy.sparse as cusparse
import pyvista as pv
import vtk
import os

from gamma.simulator.gamma import domain_mgr, heat_solve_mgr
from gamma.simulator.func import transformation

# ✅ GPU 번호 설정
cp.cuda.Device(3).use()
import os
os.chdir("/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking")

k_file_path = "/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/input_k/cylinder.k"
toolpath_file_path = "/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/toolpaths/cylinder_toolpath.crs"

domain = domain_mgr(
    filename=k_file_path,
    toolpathdir=toolpath_file_path
    )


heat_solver = heat_solve_mgr(domain)

# ✅ 물성 데이터 로딩
poisson = 0.3
a1 = 10000
young1 = cp.array(np.loadtxt("/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/examples/0_properties/TI64_Young_Debroy.txt")[:,1]/1e6)
temp_young1 = cp.array(np.loadtxt("/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/examples/0_properties/TI64_Young_Debroy.txt")[:,0])
Y1 = cp.array(np.loadtxt("/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/examples/0_properties/TI64_Yield_Debroy.txt")[:,1]/1e6*np.sqrt(2/3))
temp_Y1 = cp.array(np.loadtxt("/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/examples/0_properties/TI64_Yield_Debroy.txt")[:,0])
scl1 = cp.array(np.loadtxt("/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/examples/0_properties/TI64_Alpha_Debroy.txt")[:,1])
temp_scl1 = cp.array(np.loadtxt("/home/ftk3187/github/GAMMA/GEOASNO_research/DED_GAMMA_solver/00_masking/examples/0_properties/TI64_Alpha_Debroy.txt")[:,0])
T_Ref = domain.ambient

# ✅ 시뮬레이션 상태 초기화
n_n = len(domain.nodes)
n_e = len(domain.elements)
n_q = 8
S = cp.zeros((n_e, n_q, 6))  # 응력 텐서만 초기화

nodes_pos = domain.nodes[domain.elements]
Jac = cp.matmul(domain.Bip_ele, nodes_pos[:, cp.newaxis, :, :].repeat(8, axis=1))
ele_detJac = cp.linalg.det(Jac)

# ✅ 저장 폴더 준비
os.makedirs("results", exist_ok=True)

# ✅ 결과 저장 함수 (온도 + von Mises 응력)
def save_vtk(filename):
    n_e_save = cp.sum(domain.active_elements)
    n_n_save = cp.sum(domain.active_nodes)
    active_elements = domain.elements[domain.active_elements].tolist()
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = domain.nodes[0:n_n_save].get()

    Sv = transformation(cp.sqrt(1/2 * ((S[0:n_e_save,:,0]-S[0:n_e_save,:,1])**2 +
                                       (S[0:n_e_save,:,1]-S[0:n_e_save,:,2])**2 +
                                       (S[0:n_e_save,:,2]-S[0:n_e_save,:,0])**2 +
                                       6*(S[0:n_e_save,:,3]**2 + S[0:n_e_save,:,4]**2 + S[0:n_e_save,:,5]**2))),
                        domain.elements[0:n_e_save], ele_detJac[0:n_e_save], n_n_save)

    grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    grid.point_data["temp"] = heat_solver.temperature[0:n_n_save].get()
    grid.point_data["S_von"] = Sv.get()
    grid.save(filename)

# ✅ 메인 시뮬레이션 루프 (열전달만 수행)
for step in range(50000):
    heat_solver.time_integration()
    
    if step % 100 == 0:
        print(f"Step: {step}, Time: {domain.current_sim_time:.4f}")
        save_vtk(f"results/00_{step:04d}.vtk")


Time of reading input files: 0.44690823554992676
Time of calculating critical timestep: 0.01831364631652832
Time of reading and interpolating toolpath: 0.051604270935058594
Number of nodes: 23716
Number of elements: 20540
Number of time-steps: 100000
Time of generating surface: 0.18033385276794434
Step: 0, Time: 0.0010
Step: 100, Time: 0.1010
Step: 200, Time: 0.2010
Step: 300, Time: 0.3010
Step: 400, Time: 0.4010


KeyboardInterrupt: 